In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import json
from datetime import datetime

In [2]:
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("PIL (Pillow) available")
print("All libraries imported successfully!")

OpenCV version: 4.11.0
NumPy version: 1.26.4
PIL (Pillow) available
All libraries imported successfully!


In [3]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Created 'data' directory for storing face images")

if not os.path.exists("screenshots"):
    os.makedirs("screenshots")
    print("Created 'screenshots' directory")

In [4]:
def create_users_config():
    users_config = "users.json"

    default_users = {
        "1": "Kalhara",
        "2": "User2",
        "3": "User3"
    }

    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
        print("Loaded existing users configuration:")
    else:
        users = default_users
        with open(users_config, 'w') as f:
            json.dump(users, f, indent=2)
        print("Created new users configuration:")

    for user_id, name in users.items():
        print(f"    ID: {user_id} - Name: {name}")
    
    return users

In [5]:
def add_user(user_id, name):
    users_config = "users.json"
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
    else:
        users = {}
    users[str(user_id)] = name
    with open(users_config, 'w') as f:
        json.dump(users, f, indent=2)
    
    print(f" Added user: {name} (ID: {user_id})")
    return users
        

In [6]:
users = create_users_config()

Loaded existing users configuration:
    ID: 1 - Name: Kalhara
    ID: 2 - Name: User2
    ID: 3 - Name: User3
    ID: 4 - Name: Kalhara


In [7]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [8]:
# Optimized camera setup function
def setup_camera(camera_id=0):
    """Initialize camera with optimal settings for face recognition"""
    cap = cv2.VideoCapture(camera_id)
    
    if not cap.isOpened():
        return None
    
    # Set optimal camera properties for performance
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # Reduce buffer to minimize delay
    cap.set(cv2.CAP_PROP_FPS, 30)        # Set frame rate
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)   # Set resolution
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Test camera
    ret, frame = cap.read()
    if not ret:
        cap.release()
        return None
    
    print(f"Camera initialized: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))} @ {int(cap.get(cv2.CAP_PROP_FPS))}fps")
    return cap

In [9]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Optimize detection parameters for better performance
    faces = face_classifier.detectMultiScale(
        gray, 
        scaleFactor=1.1,     # Smaller scale factor for better accuracy
        minNeighbors=3,      # Reduced for faster detection
        minSize=(50, 50),    # Larger minimum size for better performance
        maxSize=(300, 300),  # Add maximum size to limit search area
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    if len(faces) == 0:
        return None
    
    # If multiple faces, select the largest one
    if len(faces) > 1:
        faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)

    x, y, w, h = faces[0]
    
    # Add some padding around the face
    padding = 10
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = min(img.shape[1] - x, w + 2*padding)
    h = min(img.shape[0] - y, h + 2*padding)
    
    cropped_face = img[y:y+h, x:x+w]
    
    return cropped_face
    
    
    

In [10]:
def test_face_detection():
    print("Testing face detection...")
    print(" Press 'q' to quit, 's' to save a detected face")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Cannot open camera")
        return
    
    saved_count = 0 
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read frame")
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Face {len(faces)}", (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Face Detection Test", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s') and len(faces) > 0:
            face = detect_face(frame)
            if face is not None:
                saved_count += 1
                filename = f"test_face_{saved_count}.jpg"
                cv2.imwrite(filename, face)
                print(f" Saved face as {filename}")
    cap.release()
    cv2.destroyAllWindows()
    print(f" Face detection test completed. Saved {saved_count} faces.")

In [11]:
def collect_face_data(user_id=1, num_samples=200):
    print(f" Starting data collection for User ID: {user_id}")
    print(f" Target samples: {num_samples}")
    print(" Instructions:")
    print("   - Look directly at the camera")
    print("   - Move your head slightly for different angles")
    print("   - Ensure good lighting")
    print("   - Press 'q' to quit early")

    cap = setup_camera()
    if cap is None:
        print(" Error: Cannot open camera")
        return False
    
    img_count = 0
    frame_count = 0
    capture_interval = 3
    
    print("\n Data collection started!")

    while True:
        ret, frame = cap.read()
        if not ret:
            print(" Error: Cannot read frame")
            break
        
        frame_count += 1
        display_frame = frame.copy()
        cv2.putText(display_frame, f"User ID: {user_id}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(display_frame, f"Samples: {img_count}/{num_samples}", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(display_frame, "Press 'q' to quit", (10, 90), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        if frame_count % capture_interval == 0:
            cropped_face = detect_face(frame)
            
            if cropped_face is not None:
                img_count += 1
                face_resized = cv2.resize(cropped_face, (200, 200))
                face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
                filename = f"data/user.{user_id}.{img_count}.jpg"
                cv2.imwrite(filename, face_gray)
                display_face = face_gray.copy()
                cv2.putText(display_face, f"Sample: {img_count}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                cv2.imshow("Data Collection - Processed Face", display_face)
                if img_count % 10 == 0:
                    print(f" Collected {img_count}/{num_samples} samples")

                if img_count >= num_samples:
                    print(f"\n Data collection completed!")
                    break
        cv2.imshow("Data Collection - Original", display_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        return True  

    cap.release()    
    print(f" Files saved in 'data' directory with prefix 'user.{user_id}'")
    cv2.destroyAllWindows()    
    print(f" Total samples collected: {img_count}")
    print(f"\n Data collection completed!")

In [12]:
def check_collected_data():
    print("TRAINING DATA SUMMARY")
    print("=" * 40)

    if not os.path.exists("data"):
        print("No data directory found")
        return
    
    files = [f for f in os.listdir("data") if f.endswith('.jpg')]
    
    if len(files) == 0:
        print("No training images found")
        return

    user_counts = {}
    for filename in files:
        try:
            user_id = filename.split('.')[1]
            user_counts[user_id] = user_counts.get(user_id, 0) + 1
        except:
            continue

    users_config = "users.json"
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
    else:
        users = {}

    total_samples = 0
    for user_id, count in user_counts.items():
        name = users.get(user_id, f"User {user_id}")
        print(f" {name} (ID: {user_id}): {count} samples")
        total_samples += count
    
    print(f"\n Total samples: {total_samples}")
    print(f" Users with data: {len(user_counts)}")

In [13]:
def train_face_recognizer():
    print("Starting model training...")
    if not os.path.exists("data"):
        print("Error: 'data' directory not found")
        print("Collect training data first using collect_face_data()")
        return False

    image_files = [f for f in os.listdir("data") if f.endswith('.jpg')]
    
    if len(image_files) == 0:
        print("Error: No training images found")
        print("Collect training data first using collect_face_data()")
        return False
    
    print(f" Found {len(image_files)} training images")
    faces, labels = process_training_images_batch(image_files)
    
    if len(faces) == 0:
        print("Error: No valid training images found")
        return False
    
    print(f" Successfully processed {len(faces)} images")
    print(f" Training data for {len(set(labels))} different users")
    print(" Training face recognizer model...")
    recognizer = cv2.face.LBPHFaceRecognizer_create(
        radius=1,
        neighbors=8,
        grid_x=8,
        grid_y=8
    )
    
    recognizer.train(faces, labels)
    
    model_path = "face_recognizer_model.xml"
    recognizer.write(model_path)
    
    print(f" Model training completed!")
    print(f" Model saved as: {model_path}")
    print(f"\n TRAINING SUMMARY:")
    print(f"   Total images: {len(faces)}")
    print(f"   Unique users: {len(set(labels))}")
    print(f"   Model file: {model_path}")
    
    user_counts = {}
    for label in labels:
        user_counts[label] = user_counts.get(label, 0) + 1
    
    print("\n Samples per user:")
    for user_id, count in sorted(user_counts.items()):
        print(f"   User {user_id}: {count} samples")
        
    return True

In [14]:
def process_training_images_batch(image_files, batch_size=50):
    """Process training images in batches to manage memory efficiently"""
    faces_batch = []
    labels_batch = []
    
    for i, image_file in enumerate(image_files):
        try:
            image_path = os.path.join("data", image_file)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            if img.shape != (200, 200):
                img = cv2.resize(img, (200, 200))
            user_id = int(image_file.split('.')[1])
            faces_batch.append(img)
            labels_batch.append(user_id)
            if (i + 1) % 25 == 0:
                print(f"   Processed {i + 1}/{len(image_files)} images")
                
        except Exception as e:
            print(f"  Warning: Skipping {image_file} - {e}")
            continue
    
    return np.array(faces_batch, dtype=np.uint8), np.array(labels_batch, dtype=np.int32)

In [15]:
def load_trained_model():
    model_path = "face_recognizer_model.xml"
    if not os.path.exists(model_path):
        print(f" Error: Model file '{model_path}' not found")
        print(" Train the model first using train_face_recognizer()")
        return None
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_path)
    print(f" Model loaded successfully from {model_path}")
    return recognizer

In [16]:
def validate_model():
    model_path = "face_recognizer_model.xml"
    if not os.path.exists(model_path):
        print(" No trained model found")
        return False
    model_size = os.path.getsize(model_path) / 1024 
    model_time = datetime.fromtimestamp(os.path.getmtime(model_path))
    print(" MODEL INFORMATION:")
    print(f"   File: {model_path}")
    print(f"   Size: {model_size:.1f} KB")
    print(f"   Created: {model_time.strftime('%Y-%m-%d %H:%M:%S')}")
    recognizer = load_trained_model()
    if recognizer is not None:
        print(" Model validation successful")
        return True
    else:
        print(" Model validation failed")
        return False

In [17]:
def monitor_performance():
    """Monitor system performance and provide optimization tips"""
    import time
    import psutil
    
    print("PERFORMANCE MONITORING")
    print("=" * 40)
    memory = psutil.virtual_memory()
    print(f"Available RAM: {memory.available / (1024**3):.1f} GB / {memory.total / (1024**3):.1f} GB")
    print("\\nTesting camera performance...")
    cap = setup_camera()
    if cap:
        start_time = time.time()
        frame_count = 0
        test_duration = 3 
        
        while time.time() - start_time < test_duration:
            ret, frame = cap.read()
            if ret:
                frame_count += 1
        
        fps = frame_count / test_duration
        cap.release()
        
        print(f"Camera FPS: {fps:.1f}")
        if fps < 20:
            print("Camera performance is low. Consider:")
            print("   - Close other camera applications")
            print("   - Reduce frame resolution")
            print("   - Check USB connection")
        else:
            print("Camera performance is good")
    
    if os.path.exists("data"):
        files = [f for f in os.listdir("data") if f.endswith('.jpg')]
        total_size = sum(os.path.getsize(os.path.join("data", f)) for f in files)
        print(f"\\nTraining data: {len(files)} images, {total_size / (1024**2):.1f} MB")
        
        if len(files) > 1000:
            print(" Large dataset detected. Training may be slow.")
            print("   Consider reducing samples per user to 100-200")
        
    print("\\nOptimization tips:")
    print("  - Use 100-200 samples per user for good accuracy")
    print("  - Ensure good lighting during data collection")
    print("  - Close unnecessary applications during training")

In [18]:
def load_users_config():
    users_config = "users.json"
    
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            return json.load(f)
    else:
        return {"1": "Kalhara", "2": "Unknown User"}

In [19]:
def recognize_faces(confidence_threshold=75):
    print(f" Starting face recognition...")
    print(f" Confidence threshold: {confidence_threshold}%")
    print(" Controls:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save screenshot")
    print("   - Press 'c' to change confidence threshold")

    recognizer = load_trained_model()
    if recognizer is None:
        return False

    users = load_users_config()

    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print(" Error: Cannot open camera")
        return False
    
    screenshot_count = 0
    recognition_count = 0
    
    print("\n Face recognition started!")

    while True:
        ret, frame = cap.read()
        if not ret:
            print(" Error: Cannot read frame")
            break
        display_frame = frame.copy()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(
            gray, 
            scaleFactor=1.1,
            minNeighbors=10,
            minSize=(50, 50)
        )
        
        for (x, y, w, h) in faces:
            face_roi = gray[y:y + h, x:x + w]
            user_id, confidence_score = recognizer.predict(face_roi)
            confidence = int(100 * (1 - confidence_score / 300))
            if confidence > confidence_threshold:
                user_name = users.get(str(user_id), f"User {user_id}")
                color = (0, 255, 0)  
                label = f"{user_name} ({confidence}%)"
                recognition_count += 1
            else:
                color = (0, 0, 255) 
                label = f"UNKNOWN ({confidence}%)"
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), color, 2)
            (label_width, label_height), _ = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
            cv2.rectangle(display_frame, (x, y - 35), 
                         (x + label_width, y), color, -1)
            cv2.putText(display_frame, label, (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        info_y = 30
        cv2.putText(display_frame, f"Faces: {len(faces)}", (10, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        info_y += 30
        cv2.putText(display_frame, f"Confidence: {confidence_threshold}%", (10, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        info_y += 30
        cv2.putText(display_frame, "q:quit s:screenshot c:change confidence", 
                   (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        cv2.imshow("Face Recognition System", display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('s'):
            screenshot_count += 1
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"screenshots/recognition_{timestamp}_{screenshot_count}.jpg"
            cv2.imwrite(filename, display_frame)
            print(f" Screenshot saved: {filename}")
        
        elif key == ord('c'):
            print(f"\n Current confidence threshold: {confidence_threshold}%")
            try:
                new_threshold = int(input("Enter new threshold (50-95): "))
                if 50 <= new_threshold <= 95:
                    confidence_threshold = new_threshold
                    print(f"Confidence threshold changed to: {confidence_threshold}%")
                else:
                    print("  Invalid range. Using previous value.")
            except:
                print("  Invalid input. Using previous value.")
    cap.release()
    cv2.destroyAllWindows()

    print(f"\n Face recognition session completed!")
    print(f" Total recognitions: {recognition_count}")
    print(f" Screenshots saved: {screenshot_count}")
    
    return True

In [20]:
def test_single_image(image_path):
    print(f"  Testing recognition on: {image_path}")
    recognizer = load_trained_model()
    if recognizer is None:
        return False
    users = load_users_config()
    if not os.path.exists(image_path):
        print(f" Error: Image file not found: {image_path}")
        return False
    img = cv2.imread(image_path)
    if img is None:
        print(f" Error: Cannot load image: {image_path}")
        return False
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        print(" No faces detected in the image")
        return False
    
    print(f" Found {len(faces)} face(s)")
    
    for i, (x, y, w, h) in enumerate(faces):
        face_roi = gray[y:y + h, x:x + w]
        user_id, confidence_score = recognizer.predict(face_roi)
        confidence = int(100 * (1 - confidence_score / 300))
        user_name = users.get(str(user_id), f"User {user_id}")
        print(f"  Face {i+1}: {user_name} (Confidence: {confidence}%)")
        color = (0, 255, 0) if confidence > 75 else (0, 0, 255)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{user_name} ({confidence}%)", (x, y - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.imshow("Single Image Recognition", img)
    print("👀 Press any key to close the image window")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return True

In [ ]:
def display_menu():
    print("\n" + "="*50)
    print(" FACE RECOGNITION SYSTEM")
    print("="*50)
    print("1. Add New User")
    print("2. Collect Training Data")
    print("3. Train Model")
    print("4. Start Face Recognition")
    print("5. Test Single Image")
    print("6. View System Status")
    print("7. Test Camera & Face Detection")
    print("8. Run Complete Setup Workflow")
    print("9. Monitor Performance")
    print("10. Exit")
    print("="*50)
    
def view_system_status():
    print("\n SYSTEM STATUS")
    print("="*40)
    if os.path.exists("data"):
        files = [f for f in os.listdir("data") if f.endswith('.jpg')]
        print(f" Training images: {len(files)}")
        
        user_counts = {}
        for f in files:
            try:
                user_id = f.split('.')[1]
                user_counts[user_id] = user_counts.get(user_id, 0) + 1
            except:
                continue
        
        users = load_users_config()
        for user_id, count in user_counts.items():
            name = users.get(user_id, f"User {user_id}")
            print(f"    {name}: {count} samples")
    else:
        print("No training data found")
    
    if os.path.exists("face_recognizer_model.xml"):
        model_size = os.path.getsize("face_recognizer_model.xml") / 1024
        model_time = datetime.fromtimestamp(os.path.getmtime("face_recognizer_model.xml"))
        print(f" Model: {model_size:.1f} KB")
        print(f" Last trained: {model_time.strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print(" No trained model found")
    
    users = load_users_config()
    print(f" Registered users: {len(users)}")
    for user_id, name in users.items():
        print(f"   ID {user_id}: {name}")
    
def run_complete_workflow():
    print("\nCOMPLETE SETUP WORKFLOW")
    print("="*50)
    print("This will guide you through the complete setup process:")
    print("1. User management")
    print("2. Data collection") 
    print("3. Model training")
    print("4. Testing recognition")
    print()
    
    input("Press Enter to start...")
    
    print("\nSTEP 1: USER MANAGEMENT")
    print("-" * 30)
    
    users = load_users_config()
    print("Current users:")
    for user_id, name in users.items():
        print(f"   ID {user_id}: {name}")
    
    while True:
        add_user_choice = input("\nAdd a new user? (y/n): ").lower()
        if add_user_choice != 'y':
            break
        
        try:
            new_id = input("Enter user ID (number): ")
            new_name = input("Enter user name: ")
            
            if new_id and new_name:
                add_user(int(new_id), new_name)
                users = load_users_config()
            else:
                print("Please provide both ID and name")
        except ValueError:
            print("Please enter a valid number for user ID")
    
    print("\nSTEP 2: DATA COLLECTION")
    print("-" * 30)
    
    for user_id, name in users.items():
        collect_choice = input(f"\nCollect training data for {name} (ID: {user_id})? (y/n): ").lower()
        
        if collect_choice == 'y':
            try:
                samples = input("Number of samples (default 200): ") or "200"
                samples = int(samples)
                
                print(f"\nGet ready to collect data for {name}")
                print("Position yourself in front of the camera with good lighting")
                input("Press Enter when ready...")
                
                success = collect_face_data(int(user_id), samples)
                if success:
                    print(f"Data collection completed for {name}")
                else:
                    print(f"Data collection failed for {name}")
                    
            except ValueError:
                print("Invalid number of samples")
    
    print("\nSTEP 3: MODEL TRAINING")
    print("-" * 30)
    
    train_choice = input("\nTrain the face recognition model? (y/n): ").lower()
    if train_choice == 'y':
        success = train_face_recognizer()
        if success:
            print("Model training completed successfully!")
        else:
            print("Model training failed")
    
    print("\nSTEP 4: TESTING FACE RECOGNITION")
    print("-" * 30)
    
    test_choice = input("\nTest the face recognition system? (y/n): ").lower()
    if test_choice == 'y':
        try:
            confidence = input("Confidence threshold (default 75): ") or "75"
            confidence = int(confidence)
            
            print(f"\nStarting face recognition test...")
            print("Look at the camera to test recognition")
            input("Press Enter when ready...")
            
            recognize_faces(confidence)
            
        except ValueError:
            print("Invalid confidence value, using default 75")
            recognize_faces(75)
    
    print("\nWORKFLOW COMPLETED!")
    print("Your face recognition system is ready to use!")

: 

In [ ]:
def main():
    print("Welcome to Face Recognition System!")
    print("Make sure you have a working camera connected.")
    
    while True:
        try:
            display_menu()
            choice = input("\nSelect an option (1-10): ").strip()
            
            if choice == '1':
                print("\nADD NEW USER")
                try:
                    user_id = int(input("Enter user ID (number): "))
                    name = input("Enter user name: ")
                    if name:
                        add_user(user_id, name)
                    else:
                        print("Name cannot be empty")
                except ValueError:
                    print("Please enter a valid number for user ID")
            
            elif choice == '2':
                print("\nCOLLECT TRAINING DATA")
                try:
                    user_id = int(input("Enter user ID: "))
                    samples = input("Number of samples (default 200): ") or "200"
                    samples = int(samples)
                    
                    print("\nInstructions:")
                    print("- Position yourself in front of the camera")
                    print("- Ensure good lighting")
                    print("- Look directly at the camera")
                    print("- Move your head slightly for different angles")
                    input("\nPress Enter when ready...")
                    
                    collect_face_data(user_id, samples)
                    
                except ValueError:
                    print("Please enter valid numbers")
            
            elif choice == '3':
                print("\nTRAIN MODEL")
                confirm = input("Start model training? This may take a few minutes (y/n): ").lower()
                if confirm == 'y':
                    train_face_recognizer()
            
            elif choice == '4':
                print("\nSTART FACE RECOGNITION")
                try:
                    confidence = input("Confidence threshold (default 75): ") or "75"
                    confidence = int(confidence)
                    
                    print("\nStarting face recognition...")
                    print("Controls: 'q' to quit, 's' for screenshot, 'c' to change confidence")
                    input("Press Enter when ready...")
                    
                    recognize_faces(confidence)
                    
                except ValueError:
                    print("Invalid confidence value, using default 75")
                    recognize_faces(75)
            
            elif choice == '5':
                print("\n TEST SINGLE IMAGE")
                image_path = input("Enter path to image file: ").strip()
                if image_path:
                    test_single_image(image_path)
                else:
                    print("Please provide an image path")
            
            elif choice == '6':
                view_system_status()

            elif choice == '7':
                print("\nTESTING CAMERA & FACE DETECTION")
                test_choice = input("1. Test Camera\n2. Test Face Detection\nSelect (1/2): ")
                
                if test_choice == '1':
                    print("\nCamera Test")
                    print("This will open your camera to test if it's working")
                    input("Press Enter to continue...")
                    cap = cv2.VideoCapture(0)
                    if cap.isOpened():
                        print("Camera is working!")
                        print("A window will open - press any key to close")
                        
                        ret, frame = cap.read()
                        if ret:
                            cv2.imshow("Camera Test - Press any key to close", frame)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
                        cap.release()
                    else:
                        print("Camera not accessible")
                elif test_choice == '2':
                    print("\nFace Detection Test")
                    print("This will test face detection without recognition")
                    input("Press Enter to continue...")
                    test_face_detection()
            
            elif choice == '8':
                run_complete_workflow()
            
            elif choice == '9':
                print("\nPERFORMANCE MONITORING")
                monitor_performance()
            
            elif choice == '10':
                print("\nThank you for using Face Recognition System!")
                print("Goodbye!")
                break
                
            else:
                print("Invalid option. Please select 1-10.")
            
        except KeyboardInterrupt:
            print("\n\nProgram interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\nAn error occurred: {e}")
            print("Please try again.")
if __name__ == "__main__":
    main()

Welcome to Face Recognition System!
Make sure you have a working camera connected.

 FACE RECOGNITION SYSTEM
1. Add New User
2. Collect Training Data
3. Train Model
4. Start Face Recognition
5. Test Single Image
6. View System Status
7. Test Camera & Face Detection
8. Run Complete Setup Workflow
9. Monitor Performance
10. Exit

ADD NEW USER

ADD NEW USER
 Added user: Saman (ID: 6)

 FACE RECOGNITION SYSTEM
1. Add New User
2. Collect Training Data
3. Train Model
4. Start Face Recognition
5. Test Single Image
6. View System Status
7. Test Camera & Face Detection
8. Run Complete Setup Workflow
9. Monitor Performance
10. Exit
 Added user: Saman (ID: 6)

 FACE RECOGNITION SYSTEM
1. Add New User
2. Collect Training Data
3. Train Model
4. Start Face Recognition
5. Test Single Image
6. View System Status
7. Test Camera & Face Detection
8. Run Complete Setup Workflow
9. Monitor Performance
10. Exit

COLLECT TRAINING DATA

COLLECT TRAINING DATA

Instructions:
- Position yourself in front of the 

In [ ]:
# OPTIMIZED FUNCTIONS - Use these instead of the previous versions

def optimized_collect_face_data(user_id=1, num_samples=200):
    """Optimized data collection with better performance"""
    print(f"🎯 Starting optimized data collection for User ID: {user_id}")
    print(f"📊 Target samples: {num_samples}")
    print("📋 Instructions:")
    print("   - Look directly at the camera")
    print("   - Move your head slightly for different angles")
    print("   - Ensure good lighting")
    print("   - Press 'q' to quit early")

    cap = setup_camera()
    if cap is None:
        print("❌ Error: Cannot open camera")
        return False
    
    img_count = 0
    frame_count = 0
    capture_interval = 2  # Capture every 2nd frame
    
    print("\n🚀 Optimized data collection started!")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Error: Cannot read frame")
                break
            
            frame_count += 1
            
            # Display frame with info
            display_frame = frame.copy()
            cv2.putText(display_frame, f"User ID: {user_id}", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(display_frame, f"Samples: {img_count}/{num_samples}", (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(display_frame, "Press 'q' to quit", (10, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Process every nth frame
            if frame_count % capture_interval == 0:
                cropped_face = detect_face(frame)
                
                if cropped_face is not None:
                    img_count += 1
                    
                    # Process and save
                    face_resized = cv2.resize(cropped_face, (200, 200))
                    face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
                    filename = f"data/user.{user_id}.{img_count}.jpg"
                    cv2.imwrite(filename, face_gray)
                    
                    # Progress
                    if img_count % 10 == 0:
                        print(f"✅ Collected {img_count}/{num_samples} samples")
                    
                    if img_count >= num_samples:
                        break
            
            cv2.imshow("Optimized Data Collection", display_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        cap.release()
        cv2.destroyAllWindows()
    
    print(f"\n🎉 Data collection completed!")
    print(f"📈 Total samples collected: {img_count}")
    return True

def optimized_train_face_recognizer():
    """Optimized model training with better performance"""
    print("🚀 Starting optimized model training...")
    
    if not os.path.exists("data"):
        print("❌ Error: 'data' directory not found")
        return False

    image_files = [f for f in os.listdir("data") if f.endswith('.jpg')]
    
    if len(image_files) == 0:
        print("❌ Error: No training images found")
        return False
    
    print(f"📂 Found {len(image_files)} training images")
    
    try:
        # Process images efficiently
        faces, labels = process_training_images_batch(image_files)
        
        if len(faces) == 0:
            print("❌ Error: No valid training images found")
            return False
        
        print(f"✅ Successfully processed {len(faces)} images")
        print(f"👥 Training data for {len(set(labels))} different users")
        
        # Train model
        print("🤖 Training face recognizer model...")
        recognizer = cv2.face.LBPHFaceRecognizer_create(
            radius=1, neighbors=8, grid_x=8, grid_y=8
        )
        
        recognizer.train(faces, labels)
        
        model_path = "face_recognizer_model.xml"
        recognizer.write(model_path)
        
        print(f"🎉 Model training completed!")
        print(f"💾 Model saved as: {model_path}")
        
        # Summary
        user_counts = {}
        for label in labels:
            user_counts[label] = user_counts.get(label, 0) + 1
        
        print("\n📊 TRAINING SUMMARY:")
        print(f"   Total images: {len(faces)}")
        print(f"   Unique users: {len(set(labels))}")
        print(f"   Model file: {model_path}")
        print("\n👥 Samples per user:")
        for user_id, count in sorted(user_counts.items()):
            print(f"   User {user_id}: {count} samples")
        
        return True
        
    except Exception as e:
        print(f"❌ Error during training: {e}")
        return False

# Quick performance test
def quick_performance_test():
    """Quick test to check system performance"""
    print("🔍 Quick Performance Test")
    print("=" * 30)
    
    # Test camera
    cap = setup_camera()
    if cap:
        print("✅ Camera: OK")
        cap.release()
    else:
        print("❌ Camera: Failed")
    
    # Check memory
    import psutil
    memory = psutil.virtual_memory()
    available_gb = memory.available / (1024**3)
    print(f"💾 Available RAM: {available_gb:.1f} GB")
    
    if available_gb < 2:
        print("⚠️  Warning: Low memory. Close other applications.")
    
    # Check training data
    if os.path.exists("data"):
        files = len([f for f in os.listdir("data") if f.endswith('.jpg')])
        print(f"📊 Training images: {files}")
    else:
        print("📊 Training images: 0")
    
    print("\n💡 Tips for better performance:")
    print("   - Use optimized_collect_face_data() for data collection")
    print("   - Use optimized_train_face_recognizer() for training")
    print("   - Keep 100-200 samples per user")
    print("   - Ensure good lighting during collection")

In [ ]:
# COMPREHENSIVE SYSTEM TEST AND USAGE GUIDE

def run_system_diagnostics():
    """Run comprehensive system diagnostics and provide guidance"""
    print("🔧 FACE RECOGNITION SYSTEM DIAGNOSTICS")
    print("=" * 50)
    
    # Check imports
    try:
        import cv2
        import numpy as np
        import os
        import json
        from datetime import datetime
        print("✅ All required libraries imported successfully")
    except ImportError as e:
        print(f"❌ Import error: {e}")
        return False
    
    # Check OpenCV face detection
    try:
        face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        if face_classifier.empty():
            print("❌ Face classifier not loaded")
            return False
        else:
            print("✅ Face classifier loaded successfully")
    except Exception as e:
        print(f"❌ Face classifier error: {e}")
        return False
    
    # Check directories
    dirs_to_check = ["data", "screenshots"]
    for dir_name in dirs_to_check:
        if os.path.exists(dir_name):
            print(f"✅ Directory '{dir_name}' exists")
        else:
            os.makedirs(dir_name)
            print(f"📁 Created directory '{dir_name}'")
    
    # Test camera
    cap = setup_camera()
    if cap:
        print("✅ Camera accessible")
        cap.release()
    else:
        print("❌ Camera not accessible")
    
    # Check system resources
    try:
        import psutil
        memory = psutil.virtual_memory()
        available_gb = memory.available / (1024**3)
        print(f"💾 Available RAM: {available_gb:.1f} GB")
        
        if available_gb < 2:
            print("⚠️  Warning: Low memory detected")
        else:
            print("✅ Sufficient memory available")
    except:
        print("ℹ️  Could not check system memory (psutil not available)")
    
    # Check training data
    if os.path.exists("data"):
        files = [f for f in os.listdir("data") if f.endswith('.jpg')]
        if len(files) > 0:
            print(f"📊 Found {len(files)} training images")
            
            # Analyze by user
            user_counts = {}
            for f in files:
                try:
                    user_id = f.split('.')[1]
                    user_counts[user_id] = user_counts.get(user_id, 0) + 1
                except:
                    continue
            
            for user_id, count in user_counts.items():
                status = "✅" if count >= 50 else "⚠️" if count >= 20 else "❌"
                print(f"   User {user_id}: {count} samples {status}")
        else:
            print("📊 No training images found")
    
    # Check trained model
    if os.path.exists("face_recognizer_model.xml"):
        model_size = os.path.getsize("face_recognizer_model.xml") / 1024
        print(f"🤖 Trained model: {model_size:.1f} KB")
    else:
        print("🤖 No trained model found")
    
    print("\\n🎯 RECOMMENDATIONS:")
    print("   1. Use optimized_collect_face_data() for best performance")
    print("   2. Collect 100-200 samples per user")
    print("   3. Use optimized_train_face_recognizer() for training")
    print("   4. Ensure good lighting during data collection")
    print("   5. Close other camera applications before running")
    
    return True

def quick_start_guide():
    """Display a quick start guide for new users"""
    print("🚀 QUICK START GUIDE")
    print("=" * 30)
    print("1. First time setup:")
    print("   - Run run_system_diagnostics() to check everything")
    print("   - Use option 1 to add new users")
    print("")
    print("2. Collect training data:")
    print("   - Use optimized_collect_face_data(user_id, 150)")
    print("   - Or menu option 2 for interactive collection")
    print("")
    print("3. Train the model:")
    print("   - Use optimized_train_face_recognizer()")
    print("   - Or menu option 3")
    print("")
    print("4. Start recognition:")
    print("   - Use menu option 4")
    print("   - Press 'q' to quit, 's' for screenshot")
    print("")
    print("💡 Tips:")
    print("   - Good lighting is crucial")
    print("   - Look directly at camera during collection")
    print("   - Move head slightly for variety")
    print("   - 100-200 samples per user is optimal")

# Make the system more user-friendly
def smart_menu():
    """Intelligent menu that guides users based on system state"""
    print("\\n🎯 SMART RECOMMENDATIONS")
    print("=" * 30)
    
    # Check what's available and recommend next steps
    has_users = os.path.exists("users.json")
    has_data = os.path.exists("data") and len([f for f in os.listdir("data") if f.endswith('.jpg')]) > 0
    has_model = os.path.exists("face_recognizer_model.xml")
    
    if not has_users:
        print("👤 Start by adding users (Option 1)")
    elif not has_data:
        print("📸 Next: Collect training data (Option 2)")
        print("💡 Tip: Use optimized_collect_face_data() for better performance")
    elif not has_model:
        print("🤖 Next: Train the model (Option 3)")
        print("💡 Tip: Use optimized_train_face_recognizer() for faster training")
    else:
        print("✅ System ready! You can:")
        print("   - Start face recognition (Option 4)")
        print("   - Test single images (Option 5)")
        print("   - Add more users and data as needed")
    
    print("\\n🔧 For troubleshooting, run: run_system_diagnostics()")